## Missing Values
Dataset: Home prices in Melbourne, Australia

In [1]:
import pandas as pd
price_data = pd.read_csv('Datasets/melb_data.csv')

y = price_data.Price

melb_predictors = price_data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , train_size=0.8, test_size=0.2)

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

In [3]:
cols_with_missing = [col for col in X.columns if X_train[col].isnull().any()]

reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_test = X_test.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values): ")
print(score_dataset(reduced_X_train, reduced_X_test, y_train, y_test))

MAE from Approach 1 (Drop columns with missing values): 
193919.05518970476


In [4]:
from sklearn.impute import SimpleImputer

imputed = SimpleImputer()
imputer_train = pd.DataFrame(imputed.fit_transform(X_train))
imputer_test = pd.DataFrame(imputed.transform(X_test))

print("MAE from Approach 2 (Imputation): ")
print(score_dataset(imputer_train, imputer_test, y_train, y_test))

MAE from Approach 2 (Imputation): 
185141.16578301424


In [5]:
X_train_plus = X_train.copy()
X_test_plus = X_test.copy()

for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_test_plus[col + '_was_missing'] = X_test_plus[col].isnull()

my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(imputed.fit_transform(X_train_plus))
imputed_X_test_plus = pd.DataFrame(imputed.transform(X_test_plus))

imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_test_plus.columns = X_test_plus.columns

print("MAE from Approach 3 (An Extension to Imputation): ")
print(score_dataset(imputed_X_train_plus, imputed_X_test_plus, y_train, y_test))

MAE from Approach 3 (An Extension to Imputation): 
185969.34507854687


In [6]:
X_train.shape

(10864, 12)

In [7]:
missing_val_count_by_column = X_train.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column>0])

Car               52
BuildingArea    5174
YearBuilt       4323
dtype: int64
